# InceptionV3 모델로 시도 

In [9]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.5,
    zoom_range=[0.8, 2.0],
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/train', 
    target_size=(150, 150),
    batch_size=5, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/test',
    target_size=(150,150),
    batch_size=5,
    class_mode='categorical'
)

Found 7993 images belonging to 20 classes.
Found 1999 images belonging to 20 classes.


In [13]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizer_v1 import sgd


# 합성곱 기반 층 - 이미지넷 분류기 제거 
base_model = InceptionV3(weights='imagenet', include_top=False)

# 합성곱 기반 층 출력 
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 완전연결분류기에 합성곱 기반 층 출력 주입
x = Dense(1024, activation='relu')(x)

# 분류결과 출력 층 
predictions = Dense(20, activation='softmax')(x)

# 모델 구축
model = Model(inputs=base_model.input, outputs=predictions)

# 합성곱 기반 층 가중치 동결 
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일 
model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy', 
    metrics=[keras.metrics.CategoricalAccuracy()])

# 모델 학습 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=15,
    validation_data = validation_generator, 
    validation_steps= 399, 
    #verbose=2,
    #callbacks=callbacks_list 
)

# -------- 미세조정 ---------

# 이 시점에서 상위 레이어들은 충분히 학습이 되었기에,
# inception V3의 콘볼루션 레이어에 대한 파인튜닝을 시작합니다 
# 가장 밑 N개의 레이어를 고정하고 나머지 상위 레이어를 학습시킵니다

# 레이어 이름과 레이어 인덱스를 시각화하여
# 얼마나 많은 레이어를 고정시켜야 하는지 확인합니다:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# 가장 상위 2개의 inception 블록을 학습하기로 고릅니다,
# 다시 말하면 첫 249개의 레이어는 고정시키고 나머지는 고정하지 않습니다:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# 이러한 수정사항이 효과를 내려면 모델을 다시 컴파일해야 합니다
# 낮은 학습 속도로 세팅된 SGD를 사용합니다

model.compile(
    optimizer=sgd(lr=0.0001, momentum=0.9), 
    loss='categorical_crossentropy', 
    metrics=[keras.metrics.CategoricalAccuracy()])

# 다시 한 번 모델을 학습시킵니다
# (이번엔 상위 2개의 inception 블록을 상위의 밀집 레이어들과 함께 파인튜닝합니다)
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=100,
    validation_data = validation_generator, 
    validation_steps= 399, 
    #verbose=2,
    #callbacks=callbacks_list 
)

Epoch 1/15


2022-02-17 21:35:44.820962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1589/1590 [============================>.] - ETA: 0s - loss: 2.8503 - categorical_accuracy: 0.2082

2022-02-17 21:36:30.852532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - 56s 34ms/step - loss: 2.8508 - categorical_accuracy: 0.2084 - val_loss: 2.4605 - val_categorical_accuracy: 0.2647
Epoch 2/15
1590/1590 [==============================] - 53s 34ms/step - loss: 2.4712 - categorical_accuracy: 0.2613 - val_loss: 2.3883 - val_categorical_accuracy: 0.2877
Epoch 3/15
1590/1590 [==============================] - 54s 34ms/step - loss: 2.4446 - categorical_accuracy: 0.2826 - val_loss: 2.4386 - val_categorical_accuracy: 0.3018
Epoch 4/15
1590/1590 [==============================] - 53s 33ms/step - loss: 2.4095 - categorical_accuracy: 0.2879 - val_loss: 2.6577 - val_categorical_accuracy: 0.2436
Epoch 5/15
1590/1590 [==============================] - 54s 34ms/step - loss: 2.4352 - categorical_accuracy: 0.2895 - val_loss: 2.7581 - val_categorical_accuracy: 0.2707
Epoch 6/15
1590/1590 [==============================] - 54s 34ms/step - loss: 2.4294 - categorical_accuracy: 0.2909 - val_loss: 2.6832 - val_categorical_accuracy

ValueError: `tf.compat.v1.keras` Optimizer (<keras.optimizer_v1.SGD object at 0x2b57471c0>) is not supported when eager execution is enabled. Use a `tf.keras` Optimizer instead, or disable eager execution.

In [16]:
model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0001), 
    loss='categorical_crossentropy', 
    metrics=[keras.metrics.CategoricalAccuracy()])

# 다시 한 번 모델을 학습시킵니다
# (이번엔 상위 2개의 inception 블록을 상위의 밀집 레이어들과 함께 파인튜닝합니다)
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 1590, 
    epochs=100,
    validation_data = validation_generator, 
    validation_steps= 399, 
    #verbose=2,
    #callbacks=callbacks_list 
)

Epoch 1/100


2022-02-17 21:52:39.791444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - ETA: 0s - loss: 2.6144 - categorical_accuracy: 0.1960

2022-02-17 21:53:51.269098: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1590/1590 [==============================] - 83s 49ms/step - loss: 2.6144 - categorical_accuracy: 0.1960 - val_loss: 3.0382 - val_categorical_accuracy: 0.2511
Epoch 2/100
1590/1590 [==============================] - 79s 50ms/step - loss: 2.4491 - categorical_accuracy: 0.2554 - val_loss: 2.8487 - val_categorical_accuracy: 0.3148
Epoch 3/100
1590/1590 [==============================] - 79s 50ms/step - loss: 2.3760 - categorical_accuracy: 0.2724 - val_loss: 2.7016 - val_categorical_accuracy: 0.3258
Epoch 4/100
1590/1590 [==============================] - 80s 50ms/step - loss: 2.2977 - categorical_accuracy: 0.2952 - val_loss: 2.8163 - val_categorical_accuracy: 0.3183
Epoch 5/100
1590/1590 [==============================] - 79s 50ms/step - loss: 2.2365 - categorical_accuracy: 0.3091 - val_loss: 2.6188 - val_categorical_accuracy: 0.3363
Epoch 6/100
1590/1590 [==============================] - 78s 49ms/step - loss: 2.2042 - categorical_accuracy: 0.3221 - val_loss: 2.3773 - val_categorical_acc

KeyboardInterrupt: 

# inception_V3 모델로도 35% 이상의 검증 정확도 달성하지 못했다. 
# 원인이 뭐지? <- 연구. 공부가 필요하다. 